1. Допоміжні функції:

In [1]:
import random as rnd
import hashlib as hl

def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

def reverse_mod(a, m):
    if gcd(a, m) != 1:
        raise ValueError("Modular inverse does not exist")
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += m0
    return x1

def power_mod(base, exponent, modulus):
    bin_exp = bin(exponent)[2:]  
    b = 1  
    for bit in bin_exp:
        b = (b * b) % modulus  
        if bit == '1':
            b = (b * base) % modulus  
    return b

def trial_division(n):
    if n % 2 == 0:
        return False
    for i in range(3, min(n, 1000), 2):
        if n % i == 0:
            return False
    return True

def miller_rabin_test(n, k=5):
    if n <= 1 or n == 4: return False
    if n <= 3: return True
    if not trial_division(n):  
        return False

    d = n - 1
    r = 0
    while d % 2 == 0:
        d //= 2
        r += 1

    for _ in range(k):
        a = rnd.randint(2, n - 2)
        x = power_mod(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = power_mod(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def generate_large_prime(bits):
    while True:
        num = rnd.getrandbits(bits) | (1 << (bits - 1)) | 1
        if miller_rabin_test(num):
            return num

2. Осовні високорівневі процедури:

In [49]:
def GenerateKeyPair(bits=256, e=65537):
    while True:
        p = generate_large_prime(bits)
        q = generate_large_prime(bits)
        if p == q:
            continue
        n = p * q
        phi = (p - 1) * (q - 1)
        if gcd(e, phi) == 1:
            d = reverse_mod(e, phi)
            return (e, n), (d, p, q)

def Encrypt(m, pub):
    e, n = pub
    if not (0 <= m < n):
        raise ValueError("Message out of range")
    return power_mod(m, e, n)

def Decrypt(c, priv):
    d, p, q = priv
    n = p * q
    return power_mod(c, d, n)

def Sign(message, priv):
    d, p, q = priv
    n = p * q
    
    if isinstance(message, int):
        m = message
    elif isinstance(message, str):
        m = int.from_bytes(message.encode('utf-8'), 'big')
    else:
        raise TypeError("Message must be int or str")
    
    if not (0 <= m < n):
        raise ValueError(f"Message ({m}) must be less than modulus ({n})")
    
    return power_mod(m, d, n)

def Verify(message, signature_int, pub):
    e, n = pub
    
    if isinstance(message, int):
        m = message
    elif isinstance(message, str):
        m = int.from_bytes(message.encode('utf-8'), 'big')
    else:
        raise TypeError("Message must be int or str")
    
    m_from_sig = power_mod(signature_int, e, n)
    return m == m_from_sig

def SendKey(k, sender_priv, recipient_pub):
    s = Sign(k, sender_priv)
    c = Encrypt(k, recipient_pub)
    s_encrypted = Encrypt(s, recipient_pub)
    return c, s_encrypted

def ReceiveKey(ciphertext, signature, sender_pub, recipient_priv):
    k = Decrypt(ciphertext, recipient_priv)
    s = Decrypt(signature, recipient_priv)
    if not Verify(k, s, sender_pub):
        raise ValueError("Signature verification failed")
    return k

3. Сценарій роботи:

In [ ]:
#=============================Data for testing=============================

message1 = 1234567890987654321
message2 = 9876543212345678900
message3 = "baobab"
message4 = "pytsia"
key = rnd.getrandbits(256)

A_pub, A_priv = GenerateKeyPair()
B_pub, B_priv = GenerateKeyPair()

if A_priv[1]*A_priv[2] > B_priv[1]*B_priv[2]:
    A_pub, B_pub = B_pub, A_pub
    A_priv, B_priv = B_priv, A_priv

#=============================Print Data=================================
print(f"A keys: \npub_e={A_pub[0]} \npub_n={A_pub[1]} \npriv_d={A_priv[0]} \npriv_p={A_priv[1]} \npriv_q={A_priv[2]}")
print(f"B keys: \npub_e={B_pub[0]} \npub_n={B_pub[1]} \npriv_d={B_priv[0]} \npriv_p={B_priv[1]} \npriv_q={B_priv[2]}")

#--------------message1 encrypted by A and decrypted by B--------------

ciphered = Encrypt(message1, B_pub)
decrypted = Decrypt(ciphered, B_priv)
print(f"\n----------Encrypted by A and decrypted by B:-----------\nmessage = {message1}\nEncrypted by A: {ciphered} \nDecrypted by B: {decrypted}")
print("Successful?", decrypted == message1)

#--------------message2 encrypted by B and dencrypted by A--------------

ciphered = Encrypt(message2, A_pub)
decrypted = Decrypt(ciphered, A_priv)
print(f"\n----------Encrypted by B and decrypted by A:-----------\nmessage = {message2}\nEncrypted by B: {ciphered} \nDecrypted by A: {decrypted}")
print("Successful?", decrypted == message2)

#--------------message3 signed by A and verified by B--------------

signature = Sign(message3, A_priv)
is_valid = Verify(message3, signature, A_pub)
print(f"\n----------Signed by A and verified by B:-----------\nmessage = {message3}\nSigned by A: {signature} \nVerified by B: {is_valid}")
print("Successful?", is_valid)

#--------------message4 signed by B and verified by A--------------

signature = Sign(message4, B_priv)
is_valid = Verify(message4, signature, B_pub)
print(f"\n----------Signed by B and verified by A:-----------\nmessage = {message4}\nSigned by B: {signature} \nVerified by A: {is_valid}")
print("Successful?", is_valid)

#--------------SendKey / ReceiveKey--------------

enc_key, signature = SendKey(key, A_priv, B_pub)
received_key = ReceiveKey(enc_key, signature, A_pub, B_priv)
print(f"\n----------A send key / B receive key:-----------\nkey = {key}\nSent by A: \nSignature: {signature} \nEncrypted key: {enc_key} \nReceived by B: \nReceived key: {received_key}")
print("Successful?", key == received_key)


4. Перевірка сайту:

In [ ]:
#=============================Data for testing=============================

message1 = 1234567890987654321
message2 = 9876543212345678900
message3 = "baobab"
message4 = "pytsia"
key = rnd.getrandbits(128)

my_pub, my_priv = GenerateKeyPair()
site_pub = (int("10001", 16), int("81B67C5E5D2E03326851F5B16A06094498D9E3B614EA949DC53FAB4AE6229F0347A72DFAB2F7A8142A61D6EC458738137AB6D66FA5F9778A9120B62E9DE9B239038734AC845C255638847CC90C3F1819CB56C0452581965925F744993799A7827992CBA47E0E864345956541E7ACDBB83A00B6BBFB24D484C5DAF62F844899A1", 16))

#=============================Print Data=================================
print(f"My keys: \npub=({hex(my_pub[0])[2:]}, {hex(my_pub[1])[2:]}) \npriv={my_priv}")
print(f"Site keys: \npub={site_pub}")

In [ ]:
#--------------------------Encrypt--------------------------------
print("\n------------------Encrypt------------------")
print(f"Message = {message1}")
c = Encrypt(message1, site_pub)
print(f"Encrypted: {hex(c)[2:]}")
result = int("112210F4B16C1CB1", 16)                   #СЮДА ВСТАВ ТО ШО ДАВ САЙТ!!!
print(f"Site decrypted: {result} \nEncrypt good? : {result == message1}") 


#--------------------------Decrypt----------------------------------------
print("\n------------------Decrypt------------------")
print(f"Message: {message2}")
print(f"Message bytes: {hex(message2)[2:]}")
site_encrypted_message = int("48BB70FF8DC7459A9FF9F10B666CB909CAE6568DE7285FBA559D136874260CCEB40FF68BE08F74C6D353C64D78D58C363B692EFC71A1D4B866591C9EF877FFE8", 16)
decrypted_message = Decrypt(site_encrypted_message, my_priv)

print(f"Encrypted message: {site_encrypted_message} \nDecrypted message: {decrypted_message} \nDecrypt good? {decrypted_message==message2}")

#--------------------------Sign----------------------------------------
print("\n------------------Sign--------------------")
print(f"Message = {message3}")
signature = Sign(message3, my_priv)
print(f"Signature: {hex(signature)[2:]}")

#--------------------------Verify----------------------------------------
print("\n------------------Verify------------------")
signature = int("36D2CB57B7650167BB300EF23FFC2C0BB0A4AD5C9B3583957FECF5A8779C52F6F7585AE5A6B4A53D75E5DA1C4CCAA16ACDC41D9DA2FCBCB7EDDB0242E53DCAEA", 16)
print(f"Message: {message4} \nSignature: {signature}")
is_valid = Verify(message4, signature, site_pub)
print(f"Verify good? {is_valid}")

#--------------------------SendKey----------------------------------------
print("\n------------------SendKey-----------------")
key_s = 123123123123
print(f"Key: {key_s}")
c_key, signature = SendKey(key_s, my_priv, site_pub)
print(f"Sent key: {hex(c_key)[2:]} \nSignature: {hex(signature)[2:]}")
site_received_key = int("1CAAB5C3B3", 16)
print(f"SendKey good? {site_received_key == key_s}")


#--------------------------ReceiveKey----------------------------------------
print("\n------------------ReceiveKey---------------")
key = int("4DAFD944D895C00E015DB429FF92FB101D5191D14A97272E909EF22E591038DFD3FDD5772E47FE5501380D62DA2A18E07462F647DDBCBEB1F7A099114FD3676C", 16)
signature = int("0DF18A44CDF0E161697A8BCC5813F90B73F0C1985DEECBA9F70CCBC11907ACFA8CCA626D7BD58F7AFC32DA051AF953C4AEE8191F5F605E647CB242DCD886398D", 16)
print(f"Encrypted key: {key} \nSignature: {signature}")
received_key = ReceiveKey(key, signature, site_pub, my_priv)
print(f"Received key: {received_key} \nReceiveKey good? True")

